In [133]:
from ltlcross_runner import LtlcrossRunner
from IPython.display import display, display_markdown
import pandas as pd
import spot
spot.setup()
import utils

If `rerun` is set to `False` the script uses the precomputed values. If set to `True`, all values are recomputed
(takes more time)

In [82]:
### Tools' setting ###
# PATHS
make_tgba   = 'ltl2tgba --deterministic -f %f' 
owl_version = '19.06.03'
owl_bin     = f'/usr/local/share/owl-{owl_version}/bin/'
# Tools
owl         = owl_bin+'ltl2ldgba -i %f'
nba2ldba    = make_tgba + ' --ba | '+owl_bin+'nba2ldba'
seminator   = make_tgba + ' | ./seminator'
cy          = make_tgba + ' | ./seminator --via-tba'
old         = make_tgba + ' | seminator'
# Optimizations
weak        = ' --powerset-for-weak'
entry       = ' --cut-on-SCC-entry'
levels      = ' --skip-levels'
avoid       = ' --bscc-avoid'
reuse       = ' --reuse-good-SCC'
power       = ' --powerset-on-cut'
simpl_in    = ' --simplify-input'

# Options
cut         = ' --cd'
autfilt     = ' | autfilt'
new_autfilt = ' | ../spot_dev/bin/autfilt'
simp_sd     = ' --small --tgba'
simp_cd     = ' -x simul=1,ba-simul=1 --small --tgba'
nos         = ' -s0' # disables Spot's simplifications used in Seminator
end         = ' > %O' # saves result to file

def cd(tool):
    if tool.find('owl') >= 0:
        return tool
    return tool + cut

def sd(tool):
    if tool.find('owl') >= 0:
        return tool + ' -n'
    return tool

def simpl(tool, level, simpl):
    if (tool.find('owl') < 0) and level == 'no':
        tool += nos
    if level == 'no':
        return tool
    if level == 'yes':
        return tool + autfilt + simpl
    return tool + new_autfilt + simpl

d = {
    'Sem 1,2'      : old,
    'ltl2ldba'     : owl,
    'e'            : seminator + entry,
    'es'           : seminator + entry + simpl_in,
    'esw'          : seminator + entry + simpl_in + weak,
    'esp'          : seminator + entry + simpl_in + power,
    'esa'          : seminator + entry + simpl_in + avoid,
    'esr'          : seminator + entry + simpl_in + reuse,
    'esl'          : seminator + entry + simpl_in + levels,
###
    'eswrl'        : seminator + entry + simpl_in + weak + reuse + levels,
    'eswal'        : seminator + entry + simpl_in + weak + avoid + levels,
    'eswpal'       : seminator + entry + simpl_in + weak + avoid + levels + power,
    'eswprl'       : seminator + entry + simpl_in + weak + reuse + levels + power,
###
    'eswr'        : seminator + entry + simpl_in + weak + reuse,
    'eswa'        : seminator + entry + simpl_in + weak + avoid,
    'eswpa'       : seminator + entry + simpl_in + weak + avoid + power,
    'eswpr'       : seminator + entry + simpl_in + weak + reuse + power,
    
}

### Ltlcross runner configuration ###
tools = {}
    
for (name, cmd) in d.items():
    for level in ['no','yes']:#,'new']:
        tools[f'cd.{name}.{level}'] = simpl(cd(cmd), level, simp_cd) + end
        tools[f'sd.{name}.{level}'] = simpl(sd(cmd), level, simp_sd) + end

sd_tools = [t for t in tools.keys() if t.startswith('sd')]
cd_tools = [t for t in tools.keys() if t.startswith('cd')]
    
### Numbers to measure ###
cols = ['states','edges','transitions','nondet_states','exit_status','time','acc']
tool_order = list(d.keys())

### The interface of OWL has changed (without documention) and it seems it can only produce cut-deterministic automata now! :-(

In [103]:
rerun = False

In [84]:
runners = {}
for source in ('literature','random'):
    for t in ['nd']:
        name = '{}_{}'.format(source,t)
        runners[name] = \
            LtlcrossRunner(tools,\
                    res_filename='data/{}.csv'.format(name),\
                    formula_files=['formulae/{}.ltl'.format(name)],\
                    cols=cols)
        if rerun:
            runners[name].run_ltlcross(timeout='120',check=False)
        runners[name].parse_results()

In [85]:
def get_counts(runner):
    v = r.values.states.copy()
    v.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in v.columns])
    return v.cd.dropna().count()[0], v.sd.dropna().count()[0]

## Cummulative results

Gather the data into one big DataFrame

In [86]:
data = pd.DataFrame()
for (name,r) in runners.items():
    # Divide into sd_tools & cd_tools to treat timeouts separately.
    tmp_cd = pd.DataFrame(r.cummulative(tools=cd_tools),columns=[name])
    tmp_sd = pd.DataFrame(r.cummulative(tools=sd_tools),columns=[name])
    data = data.append(tmp_cd.append(tmp_sd).transpose())
data

tool,"cd.Sem 1,2.no","cd.Sem 1,2.yes",cd.ltl2ldba.no,cd.ltl2ldba.yes,cd.e.no,cd.e.yes,cd.es.no,cd.es.yes,cd.esw.no,cd.esw.yes,...,sd.eswprl.no,sd.eswprl.yes,sd.eswr.no,sd.eswr.yes,sd.eswa.no,sd.eswa.yes,sd.eswpa.no,sd.eswpa.yes,sd.eswpr.no,sd.eswpr.yes
literature_nd,603,375,440,371,602,369,598,369,597,369,...,369,282,403,303,404,303,373,285,372,285
random_nd,1995,1062,1077,806,1942,936,1898,939,1894,939,...,861,696,913,724,916,729,888,718,885,714


Get the number of formulas such that all tools finished within timeout for each category.

In [87]:
counts = pd.DataFrame(index=pd.Index([],name='source'),columns=['cd..n','sd..n'])
for (name,r) in runners.items():
    cd_c, sd_c = get_counts(r)
    counts = counts.append(pd.DataFrame({'cd..n':cd_c,'sd..n':sd_c},index=[name]))
counts

,cd..n,sd..n
literature_nd,23,23
random_nd,100,100


Converts the DataFrame to use MultiIndices for rows and columns

In [88]:
from pandas.api.types import CategoricalDtype
automata_type = CategoricalDtype(categories=['det','cd','sd','nd'], ordered=True)
source_type = CategoricalDtype(categories=['random', 'literature', 'rand. (T/O)', 'lit. (T/O)'], ordered=True)

In [89]:
def make_hierarchical(data):
    # Split index into origin & ltl2tgba's output type
    df = data.copy()
    df.index.name = 'origin'
    df.reset_index(inplace=True)
    df['type'] = df.origin.apply(lambda x: x.split('_')[1]).astype(automata_type)
    df['origin'] = df.origin.apply(lambda x: x.split('_')[0]).astype(source_type)
    df = df.set_index(['origin','type']).sort_index()
    # Split columns in tool & reductions
    df.columns = pd.MultiIndex.from_tuples([tuple(c.split('.')) for c in df.columns])
    return df

In [90]:
tmp = make_hierarchical(data.join(counts)).sort_index(axis=1)
tmp

cd                                                       \
                     Sem 1,2           e         es        esa        esl   
                   n      no   yes    no  yes    no  yes    no  yes    no   
origin     type                                                             
random     nd    100    1995  1062  1942  936  1898  939  1040  788  1863   
literature nd     23     603   375   602  369   598  369   487  359   594   

                 ...    sd                                                      
                 ... eswpr      eswprl      eswr      eswrl      ltl2ldba       
                 ...    no  yes     no  yes   no  yes    no  yes       no  yes  
origin     type  ...                                                            
random     nd    ...   885  714    861  696  913  724   892  695     1077  792  
literature nd    ...   372  285    369  282  403  303   400  300      440  367  

[2 rows x 70 columns]

In [91]:
tmp.cd

Sem 1,2           e         es        esa        esl  \
                   n      no   yes    no  yes    no  yes    no  yes    no   
origin     type                                                             
random     nd    100    1995  1062  1942  936  1898  939  1040  788  1863   
literature nd     23     603   375   602  369   598  369   487  359   594   

                 ... eswpr      eswprl       eswr      eswrl      ltl2ldba  \
                 ...    no  yes     no  yes    no  yes    no  yes       no   
origin     type  ...                                                         
random     nd    ...  1003  784    979  766  1037  782  1010  753     1077   
literature nd    ...   455  325    452  325   486  359   483  355      440   

                      
                 yes  
origin     type       
random     nd    806  
literature nd    371  

[2 rows x 35 columns]

In [92]:
cd_y = tmp.cd.swaplevel(axis=1).yes
cd_y

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1062,936,939,788,907,934,782,939,788,757,788,770,784,766,782,753,806
literature,nd,375,369,369,359,365,335,359,369,359,355,325,325,325,325,359,355,371


In [93]:
tmp.cd.swaplevel(axis=1).no

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1995,1942,1898,1040,1863,1865,1040,1894,1040,1013,1006,982,1003,979,1037,1010,1077
literature,nd,603,602,598,487,594,567,486,597,487,483,456,452,455,452,486,483,440


In [94]:
tmp.sd.swaplevel(axis=1).yes

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,789,735,735,729,704,724,724,735,729,699,718,701,714,696,724,695,792
literature,nd,328,309,309,303,306,290,303,309,303,300,285,282,285,282,303,300,367


In [95]:
tmp.sd.swaplevel(axis=1).no

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1178,1122,1049,916,1025,1020,916,1047,916,895,888,864,885,861,913,892,1077
literature,nd,431,428,424,404,420,393,403,423,404,400,373,369,372,369,403,400,440


In [96]:
tmp.sd.eswprl

,,no,yes
origin,type,,
random,nd,861,696
literature,nd,369,282


## Timeouts

In [97]:
TO_data = pd.DataFrame()
for (name,r) in runners.items():
    TO_data = TO_data.append(pd.DataFrame(r.compute_timeouts(),columns=[name]).transpose())
TO = make_hierarchical(TO_data).replace(0,pd.NaT).dropna(how='all').fillna(0)
TO

Empty DataFrame
Columns: [(cd, Sem 1,2, no), (cd, Sem 1,2, yes), (cd, e, no), (cd, e, yes), (cd, es, no), (cd, es, yes), (cd, esa, no), (cd, esa, yes), (cd, esl, no), (cd, esl, yes), (cd, esp, no), (cd, esp, yes), (cd, esr, no), (cd, esr, yes), (cd, esw, no), (cd, esw, yes), (cd, eswa, no), (cd, eswa, yes), (cd, eswal, no), (cd, eswal, yes), (cd, eswpa, no), (cd, eswpa, yes), (cd, eswpal, no), (cd, eswpal, yes), (cd, eswpr, no), (cd, eswpr, yes), (cd, eswprl, no), (cd, eswprl, yes), (cd, eswr, no), (cd, eswr, yes), (cd, eswrl, no), (cd, eswrl, yes), (cd, ltl2ldba, no), (cd, ltl2ldba, yes), (sd, Sem 1,2, no), (sd, Sem 1,2, yes), (sd, e, no), (sd, e, yes), (sd, es, no), (sd, es, yes), (sd, esa, no), (sd, esa, yes), (sd, esl, no), (sd, esl, yes), (sd, esp, no), (sd, esp, yes), (sd, esr, no), (sd, esr, yes), (sd, esw, no), (sd, esw, yes), (sd, eswa, no), (sd, eswa, yes), (sd, eswal, no), (sd, eswal, yes), (sd, eswpa, no), (sd, eswpa, yes), (sd, eswpal, no), (sd, eswpal, yes), (sd, eswpr, no), (sd, eswpr, yes), (sd, eswprl, no), (sd, eswprl, yes), (sd, eswr, no), (sd, eswr, yes), (sd, eswrl, no), (sd, eswrl, yes), (sd, ltl2ldba, no), (sd, ltl2ldba, yes)]
Index: []

[0 rows x 68 columns]

# Final results
We use the `display` function to propagate the results to notebooks that just run this one.

In [98]:
display_markdown('''### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.

- **e:** `cut-on-scc-entry` cut transition also for all transitions that go from SCC S to S' where S' is accepting, see [Cut-on-SCC-entry section of Cut-edges notebook](options_notebooks/Cut-edges.ipynb#Cut-on-SCC-entry)
- **s:** `simplify input`
- **w:** `powerset-for-weak` do not run breakpoint construction on SCCs that are inherently weak, use powerset instead
- **p:** `powerset-on-cut` when creating cut transition, run breakpoint/powerset construction already on the cut-transition, see [Section powerset-on-cut of Cut-edges notebook](options_notebooks/Cut-edges.ipynb#powerset-on-cut)
- **l:** `level-skipping` try to reach breakpoint for as many levels as possible, for an example see the [Level skipping notebook](options_notebooks/LevelSkipping.ipynb)
- **a:** `avoid-det-tails` avoid determinsitic tails in the 1st component. An SCC is a deterministic tail if and only if it is deterministic and no non-deterministic state is reachable from it. See the [Deterministic tails notebook](options_notebooks/bSCC.ipynb)
- **r:** `reuse-det-tails` avoid determinsitic tails in the 1st component and reuse them as they are (no breakpoint, TGBA acceptance). See more details [here](options_notebooks/bSCC.ipynb#Reusing-the-semi-deterministic-components-with-TGBA-acceptance).

#### Without simplifications
''',raw=True)

### Comparison of tools producing cut-deterministic automata
All tools finished within the one-minute time limit.

- **e:** `cut-on-scc-entry` cut transition also for all transitions that go from SCC S to S' where S' is accepting, see [Cut-on-SCC-entry section of Cut-edges notebook](options_notebooks/Cut-edges.ipynb#Cut-on-SCC-entry)
- **s:** `simplify input`
- **w:** `powerset-for-weak` do not run breakpoint construction on SCCs that are inherently weak, use powerset instead
- **p:** `powerset-on-cut` when creating cut transition, run breakpoint/powerset construction already on the cut-transition, see [Section powerset-on-cut of Cut-edges notebook](options_notebooks/Cut-edges.ipynb#powerset-on-cut)
- **l:** `level-skipping` try to reach breakpoint for as many levels as possible, for an example see the [Level skipping notebook](options_notebooks/LevelSkipping.ipynb)
- **a:** `avoid-det-tails` avoid determinsitic tails in the 1st component. An SCC is a deterministic tail if and only if it is deterministic and no non-deterministic state is reachable from it. See the [Deterministic tails notebook](options_notebooks/bSCC.ipynb)
- **r:** `reuse-det-tails` avoid determinsitic tails in the 1st component and reuse them as they are (no breakpoint, TGBA acceptance). See more details [here](options_notebooks/bSCC.ipynb#Reusing-the-semi-deterministic-components-with-TGBA-acceptance).

#### Without simplifications


In [99]:
display(tmp.cd.swaplevel(axis=1).no)

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1995,1942,1898,1040,1863,1865,1040,1894,1040,1013,1006,982,1003,979,1037,1010,1077
literature,nd,603,602,598,487,594,567,486,597,487,483,456,452,455,452,486,483,440


In [100]:
display_markdown('#### With simplifications', raw=True)

#### With simplifications

In [101]:
display(tmp.cd.swaplevel(axis=1).yes)

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1062,936,939,788,907,934,782,939,788,757,788,770,784,766,782,753,806
literature,nd,375,369,369,359,365,335,359,369,359,355,325,325,325,325,359,355,371


In [102]:
display_markdown('''### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications
''',raw=True)

### Comparison of tools producing semi-deterministic automata
All tools finished within the one-minute time limit.

#### Without simplifications


In [104]:
display(tmp.sd.swaplevel(axis=1).no)

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1178,1122,1049,916,1025,1020,916,1047,916,895,888,864,885,861,913,892,1077
literature,nd,431,428,424,404,420,393,403,423,404,400,373,369,372,369,403,400,440


In [105]:
display_markdown('#### With simplifications', raw=True)

#### With simplifications

In [106]:
display(tmp.sd.swaplevel(axis=1).yes)

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,789,735,735,729,704,724,724,735,729,699,718,701,714,696,724,695,792
literature,nd,328,309,309,303,306,290,303,309,303,300,285,282,285,282,303,300,367


In [79]:
r.values.states[['cd.ltl2ldba.no','cd.eswprl.yes']].sum()

tool
cd.ltl2ldba.no    1077
cd.eswprl.yes      766
dtype: int64

In [ ]:
todo = runners['random_nd'].smaller_than('cd.ltl2ldba.no','cd.eswprl.yes')
#print(todo.size)
todo

In [ ]:
todo = runners['random_nd'].smaller_than('cd.ltl2ldba.yes','cd.eswprl.yes')
#print(todo.size)
todo

In [ ]:
r = runners['random_nd']
f = r.form_of_id(10)
spot.translate(f)

In [ ]:
r.aut_for_id(10, 'cd.ltl2ldba.yes')

In [ ]:
a = r.aut_for_id(10, 'cd.eswprl.yes')
spot.highlight_nondet_states(a, 2); a

In [ ]:
rr = runners['random_nd']
reuse = "cd.eswprl"
avoid = "cd.eswpal"
diff = rr.smaller_than(f'{avoid}.yes',f'{reuse}.yes')
#print(diff.size)
diff

In [ ]:
rr.aut_for_id(0,f'{reuse}.no')

In [ ]:
list(todo.index.droplevel().values)

In [ ]:
spot.translate(rr.form_of_id(7))

In [107]:
cd_y

,,"Sem 1,2",e,es,esa,esl,esp,esr,esw,eswa,eswal,eswpa,eswpal,eswpr,eswprl,eswr,eswrl,ltl2ldba
origin,type,,,,,,,,,,,,,,,,,
random,nd,1062,936,939,788,907,934,782,939,788,757,788,770,784,766,782,753,806
literature,nd,375,369,369,359,365,335,359,369,359,355,325,325,325,325,359,355,371


In [135]:
display_markdown("""## Effect of optimizations on cut-det automata after simplifications

- effect of `w` eliminated completely by simplifications,
- `avoid` and `reuse` have significant effect
""",raw=True)

## Effect of optimizations on cut-det automata after simplifications

- effect of `w` eliminated completely by simplifications,
- `avoid` and `reuse` have significant effect


In [136]:
display(cd_y.loc(axis=1)[['Sem 1,2','e','es', 'esw','esl','esa','eswa','eswr']])

,,"Sem 1,2",e,es,esw,esl,esa,eswa,eswr
origin,type,,,,,,,,
random,nd,1062,936,939,939,907,788,788,782
literature,nd,375,369,369,369,365,359,359,359


In [137]:
display_markdown("""
### Effect of `powerset-on-cut`
""",raw=True)
display(cd_y.loc(axis=1)[['eswa', 'eswpa','eswr','eswpr']])


### Effect of `powerset-on-cut`


,,eswa,eswpa,eswr,eswpr
origin,type,,,,
random,nd,788,788,782,784
literature,nd,359,325,359,325


In [138]:
display_markdown("""
### Effect of `level-skipping`
""",raw=True)
display(cd_y.loc(axis=1)[['eswpa','eswpal','eswpr','eswprl']])


### Effect of `level-skipping`


,,eswpa,eswpal,eswpr,eswprl
origin,type,,,,
random,nd,788,770,784,766
literature,nd,325,325,325,325


In [140]:
display_markdown("""
### Comparison of suggested default configurations with other tools
""",raw=True)
display(cd_y.loc(axis=1)[['Sem 1,2','eswrl','eswprl','ltl2ldba']])


### Comparison of suggested default configurations with other tools


,,"Sem 1,2",eswrl,eswprl,ltl2ldba
origin,type,,,,
random,nd,1062,753,766,806
literature,nd,375,355,325,371
